
<a id='missing-data'></a>
{{ header }}

# Working with missing data

In this section, we will discuss missing (also referred to as NA) values in
pandas.

>**Note**
>
>The choice of using `NaN` internally to denote missing data was largely
for simplicity and performance reasons. It differs from the MaskedArray
approach of, for example, `scikits.timeseries`. We are hopeful that
NumPy will soon be able to provide a native NA type solution (similar to R)
performant enough to be used in pandas.

See the [cookbook](cookbook.ipynb#cookbook-missing-data) for some advanced strategies.

## Values considered “missing”

As data comes in many shapes and forms, pandas aims to be flexible with regard
to handling missing data. While `NaN` is the default missing value marker for
reasons of computational speed and convenience, we need to be able to easily
detect this value with data of different types: floating point, integer,
boolean, and general object. In many cases, however, the Python `None` will
arise and we wish to also consider that “missing” or “not available” or “NA”.

>**Note**
>
>If you want to consider `inf` and `-inf` to be “NA” in computations,
you can set `pandas.options.mode.use_inf_as_na = True`.


<a id='missing-isna'></a>
To make detecting missing values easier (and across different array dtypes),
pandas provides the `isna()` and
`notna()` functions, which are also methods on
Series and DataFrame objects:

One has to be mindful that in Python (and NumPy), the `nan's` don’t compare equal, but `None's` **do**.
Note that pandas/NumPy uses the fact that `np.nan != np.nan`, and treats `None` like `np.nan`.

So as compared to above, a scalar equality comparison versus a `None/np.nan` doesn’t provide useful information.

### Integer dtypes and missing data

Because `NaN` is a float, a column of integers with even one missing values
is cast to floating-point dtype (see [Support for integer NA](gotchas.ipynb#gotchas-intna) for more). Pandas
provides a nullable integer array, which can be used by explicitly requesting
the dtype:

Alternatively, the string alias `dtype='Int64'` (note the capital `"I"`) can be
used.

See [Nullable integer data type](integer_na.ipynb#integer-na) for more.

### Datetimes

For datetime64[ns] types, `NaT` represents missing values. This is a pseudo-native
sentinel value that can be represented by NumPy in a singular dtype (datetime64[ns]).
pandas objects provide compatibility between `NaT` and `NaN`.


<a id='missing-inserting'></a>

### Inserting missing data

You can insert missing values by simply assigning to containers. The
actual missing value used will be chosen based on the dtype.

For example, numeric containers will always use `NaN` regardless of
the missing value type chosen:

Likewise, datetime containers will always use `NaT`.

For object containers, pandas will use the value given:

### Calculations with missing data

Missing values propagate naturally through arithmetic operations between pandas
objects.

The descriptive statistics and computational methods discussed in the
[data structure overview](getting_started/basics.ipynb#basics-stats) (and listed here and here) are all written to
account for missing data. For example:

- When summing data, NA (missing) values will be treated as zero.  
- If the data are all NA, the result will be 0.  
- Cumulative methods like `cumsum()` and `cumprod()` ignore NA values by default, but preserve them in the resulting arrays. To override this behaviour and include NA values, use `skipna=False`.  



<a id='missing-data-numeric-sum'></a>

## Sum/prod of empties/nans

This behavior is now standard as of v0.22.0 and is consistent with the default in `numpy`; previously sum/prod of all-NA or empty Series/DataFrames would return NaN.
See [v0.22.0 whatsnew](whatsnew/v0.22.0.ipynb#whatsnew-0220) for more.

The sum of an empty or all-NA Series or column of a DataFrame is 0.

The product of an empty or all-NA Series or column of a DataFrame is 1.

## NA values in GroupBy

NA groups in GroupBy are automatically excluded. This behavior is consistent
with R, for example:

See the groupby section [here](groupby.ipynb#groupby-missing) for more information.

### Cleaning / filling missing data

pandas objects are equipped with various data manipulation methods for dealing
with missing data.


<a id='missing-data-fillna'></a>

## Filling missing values: fillna

`fillna()` can “fill in” NA values with non-NA data in a couple
of ways, which we illustrate:

**Replace NA with a scalar value**

**Fill gaps forward or backward**

Using the same filling arguments as [reindexing](getting_started/basics.ipynb#basics-reindexing), we
can propagate non-NA values forward or backward:


<a id='missing-data-fillna-limit'></a>
**Limit the amount of filling**

If we only want consecutive gaps filled up to a certain number of data points,
we can use the limit keyword:

To remind you, these are the available filling methods:

|Method|Action|
|:----------------------------:|:------------------------------------------------:|
|pad / ffill|Fill values forward|
|bfill / backfill|Fill values backward|
With time series data, using pad/ffill is extremely common so that the “last
known value” is available at every time point.

`ffill()` is equivalent to `fillna(method='ffill')`
and `bfill()` is equivalent to `fillna(method='bfill')`


<a id='missing-data-pandasobject'></a>

## Filling with a PandasObject

You can also fillna using a dict or Series that is alignable. The labels of the dict or index of the Series
must match the columns of the frame you wish to fill. The
use case of this is to fill a DataFrame with the mean of that column.

Same result as above, but is aligning the ‘fill’ value which is
a Series in this case.


<a id='missing-data-dropna'></a>

## Dropping axis labels with missing data: dropna

You may wish to simply exclude labels from a data set which refer to missing
data. To do this, use `dropna()`:

An equivalent `dropna()` is available for Series.
DataFrame.dropna has considerably more options than Series.dropna, which can be
examined in the API.


<a id='missing-data-interpolate'></a>

## Interpolation

New in version 0.23.0: The `limit_area` keyword argument was added.

Both Series and DataFrame objects have `interpolate()`
that, by default, performs linear interpolation at missing data points.

Index aware interpolation is available via the `method` keyword:

For a floating-point index, use `method='values'`:

You can also interpolate with a DataFrame:

The `method` argument gives access to fancier interpolation methods.
If you have [scipy](http://www.scipy.org) installed, you can pass the name of a 1-d interpolation routine to `method`.
You’ll want to consult the full scipy interpolation [documentation](http://docs.scipy.org/doc/scipy/reference/interpolate.html#univariate-interpolation) and reference [guide](http://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html) for details.
The appropriate interpolation method will depend on the type of data you are working with.

- If you are dealing with a time series that is growing at an increasing rate,
  `method='quadratic'` may be appropriate.  
- If you have values approximating a cumulative distribution function,
  then `method='pchip'` should work well.  
- To fill missing values with goal of smooth plotting, consider `method='akima'`.  


These methods require `scipy`.

When interpolating via a polynomial or spline approximation, you must also specify
the degree or order of the approximation:

Compare several methods:

Another use case is interpolation at *new* values.
Suppose you have 100 observations from some distribution. And let’s suppose
that you’re particularly interested in what’s happening around the middle.
You can mix pandas’ `reindex` and `interpolate` methods to interpolate
at the new values.


<a id='missing-data-interp-limits'></a>

### Interpolation limits

Like other pandas fill methods, `interpolate()` accepts a `limit` keyword
argument. Use this argument to limit the number of consecutive `NaN` values
filled since the last valid observation:

By default, `NaN` values are filled in a `forward` direction. Use
`limit_direction` parameter to fill `backward` or from `both` directions.

By default, `NaN` values are filled whether they are inside (surrounded by)
existing valid values, or outside existing valid values. Introduced in v0.23
the `limit_area` parameter restricts filling to either inside or outside values.


<a id='missing-data-replace'></a>

## Replacing generic values

Often times we want to replace arbitrary values with other values.

`replace()` in Series and `replace()` in DataFrame provides an efficient yet
flexible way to perform such replacements.

For a Series, you can replace a single value or a list of values by another
value:

You can replace a list of values by a list of other values:

You can also specify a mapping dict:

For a DataFrame, you can specify individual values by column:

Instead of replacing with specified values, you can treat all given values as
missing and interpolate over them:


<a id='missing-data-replace-expression'></a>

## String/regular expression replacement

>**Note**
>
>Python strings prefixed with the `r` character such as `r'hello world'`
are so-called “raw” strings. They have different semantics regarding
backslashes than strings without this prefix. Backslashes in raw strings
will be interpreted as an escaped backslash, e.g., `r'\' == '\\'`. You
should [read about them](https://docs.python.org/3/reference/lexical_analysis.html#string-literals)
if this is unclear.

Replace the ‘.’ with `NaN` (str -> str):

Now do it with a regular expression that removes surrounding whitespace
(regex -> regex):

Replace a few different values (list -> list):

list of regex -> list of regex:

Only search in column `'b'` (dict -> dict):

Same as the previous example, but use a regular expression for
searching instead (dict of regex -> dict):

You can pass nested dictionaries of regular expressions that use `regex=True`:

Alternatively, you can pass the nested dictionary like so:

You can also use the group of a regular expression match when replacing (dict
of regex -> dict of regex), this works for lists as well.

You can pass a list of regular expressions, of which those that match
will be replaced with a scalar (list of regex -> regex).

All of the regular expression examples can also be passed with the
`to_replace` argument as the `regex` argument. In this case the `value`
argument must be passed explicitly by name or `regex` must be a nested
dictionary. The previous example, in this case, would then be:

This can be convenient if you do not want to pass `regex=True` every time you
want to use a regular expression.

>**Note**
>
>Anywhere in the above `replace` examples that you see a regular expression
a compiled regular expression is valid as well.

## Numeric replacement

`replace()` is similar to `fillna()`.

Replacing more than one value is possible by passing a list.

You can also operate on the DataFrame in place:

When replacing multiple `bool` or `datetime64` objects, the first
argument to `replace` (`to_replace`) must match the type of the value
being replaced. For example,

In [ ]:
>>> s = pd.Series([True, False, True])
>>> s.replace({'a string': 'new value', True: False})  # raises
TypeError: Cannot compare types 'ndarray(dtype=bool)' and 'str'

will raise a `TypeError` because one of the `dict` keys is not of the
correct type for replacement.

However, when replacing a *single* object such as,

the original `NDFrame` object will be returned untouched. We’re working on
unifying this API, but for backwards compatibility reasons we cannot break
the latter behavior. See [GH6354](https://github.com/pandas-dev/pandas/issues/6354) for more details.

### Missing data casting rules and indexing

While pandas supports storing arrays of integer and boolean type, these types
are not capable of storing missing data. Until we can switch to using a native
NA type in NumPy, we’ve established some “casting rules”. When a reindexing
operation introduces missing data, the Series will be cast according to the
rules introduced in the table below.

|data type|Cast to|
|:--------------------------------------:|:--------------------------------------:|
|integer|float|
|boolean|object|
|float|no cast|
|object|no cast|
For example:

Ordinarily NumPy will complain if you try to use an object array (even if it
contains boolean values) instead of a boolean array to get or set values from
an ndarray (e.g. selecting values based on some criteria). If a boolean vector
contains NAs, an exception will be generated:

However, these can be filled in using `fillna()` and it will work fine:

Pandas provides a nullable integer dtype, but you must explicitly request it
when creating the series or column. Notice that we use a capital “I” in
the `dtype="Int64"`.

See [Nullable integer data type](integer_na.ipynb#integer-na) for more.